In [12]:
#! pip install ipywidgets
#! jupyter nbextension enable --py widgetsnbextension --sys-prefix
from ipywidgets import interact
from ipywidgets.widgets import FloatText, BoundedIntText, FloatSlider, IntRangeSlider

import numpy as np
import matplotlib.pyplot as plt

from typing import Tuple

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [9]:
# Widgets for interactive functions / plots
deposit_widget = FloatText(value=1000, step=10, description='Deposit:')
interest_widget = FloatSlider(value=0.05, step=0.01, min=0, max=0.25, description='Interest:')
periods_widget = BoundedIntText(value=1, step=1, min=0, description='Periods:')
periods_range_widget = IntRangeSlider(value=(70, 90), step=1, min=1, max=100, description='Periods:')
frequency_widget = BoundedIntText(value=1, step=1, min=1, description='Frequency:')

#### Compound interest

Compound interest includes interest earned on the interest that was previously accumulated. Let $D_0$ be initial deposit, $p\%$ periodic interest rate and $D_n$ deposit amount after $n$ periods. Then:

\begin{cases}
    D_0 & = D_0 \\
    D_1 & = D_0 + D_0 \cdot p\% = D_0 (1 + p\%) \\
    D_2 & = D_1 + D_1 \cdot p\% = D_1 (1 + p\%) \\
    \vdots \\
    D_n & = D_{n-1} + D_{n-1} \cdot p\% = D_{n-1} (1 + p\%)
\end{cases}

Given this set of recursive equations we can compute the deposit amount after $n$ periods.

In [3]:
def D_(n: int, D_0: float, p: float):
    if n == 0:
        return D_0
    else:
        return D_(n-1, D_0, p) * (1 + p)

D_10 = D_(n=10, D_0=1000, p=0.05)
print(f'Initial deposit 1000 after 10 years with 5% interest: {D_10:.2f}')

Initial deposit 1000 after 10 years with 5% interest: 1628.89


### Closed formula for compound interest
Given set of equations we can try to find closed formula for compound interest:

\begin{cases}
    D_0 & = D_0 (1 + p\%)^0\\
    D_1 & = D_0 (1 + p\%) = D_0 (1 + p\%)^1 \\
    D_2 & = D_1 (1 + p\%) = D_0 (1 + p\%)^2 \\
    D_3 & = D_2 (1 + p\%) = D_0 (1 + p\%)^3 \\
    \vdots
\end{cases}
so our conjecture is that $D_n = D_0 (1 + p\%)^n$

##### Theorem
The deposit amount, given initial deposit $D_0$ after $n$ periods with $p\%$ periodic interest rate, i.e. $D_n$ is equal to $D_0 (1 + p\%)^n$.

##### Proof by induction
For $n=1$ we have: $D_1 = D_0 (1+p\%)^1$ by definition.

IH: $D_n = D_0 (1 + p\%)^n$

IS: $D_{n+1} = D_n (1 + p\%) = D_0 (1+p\%)^n (1 + p\%) = D_0 (1+p\%)^{n+1}$ and that finishes the proof.

In [4]:
def deposit_compound(n: int, D_0: float, p: float):
    """Compute recursively deposit amount after n periods.

    Args:
        n (int): period
        D_0 (float): initial deposit
        p (float in [0,1]): annual interest rate

    Returns:
        D_n (float): deposit amount after n periods.
    """
    return D_0 * (1 + p)**n

D_10 = deposit_compound(n=10, D_0=1000, p=0.05)
print(f'Initial deposit 1000 after 10 years with 5% interest: {D_10:.2f}')
print('Is closed formula correct?', abs(D_10 - D_(10, 1000, 0.05)) < 0.001)

Initial deposit 1000 after 10 years with 5% interest: 1628.89
Is closed formula correct? True


### Compound interest calculator

In [5]:
def pretty_print_deposit(n, D_0, p):
    return f'Deposit after {n} periods with {p} interest: {deposit_compound(n, D_0, p):.2f}'
interact(pretty_print_deposit, n=periods_widget, D_0=deposit_widget, p=interest_widget);

interactive(children=(BoundedIntText(value=1, description='Periods:'), FloatText(value=1000.0, description='De…

### Compound frequency
The compounding frequency is the number of times per period the accumulated interest is paid out, or capitalized. Periodic interest rate is divided then by the number of compounding frequency. Let $D_0$ be initial deposit, $p\%$ periodic interest rate, $m$ frequency of compounding and denote $D_{n,k}, \; k=0,\ldots, m$ as deposit amount in $n$-th period and $k$-th compounding. Also for convenience denote $D_n = D_{n,0} = D_{n-1, m}, \; D_{0,0} = D_0$ as deposit after $n$ periods. Let us start with $m=2$ then:

\begin{cases}
    D_{0,0} & = D_0 \\
    D_{0,1} & = D_{0,0} + D_{0,0} \cdot \frac{p}{2}\% = D_{0,0} \left( 1 + \frac{p}{2}\% \right) \\
    D_{1,0} & = D_{0,1} + D_{0,1} \cdot \frac{p}{2}\% = D_{0,1} \left( 1 + \frac{p}{2}\% \right) \\
    D_{1,1} & = D_{1,0} + D_{1,0} \cdot \frac{p}{2}\% = D_{1,0} \left( 1 + \frac{p}{2}\% \right) \\
    D_{2,0} & = D_{1,1} + D_{1,1} \cdot \frac{p}{2}\% = D_{1,1} \left( 1 + \frac{p}{2}\% \right) \\
    \vdots \\
    D_{n-1,1} & = D_{n-1,0} + D_{n-1,0} \cdot \frac{p}{2}\% = D_{n-1,0} \left( 1 + \frac{p}{2}\% \right) \\
    D_{n,0} & = D_{n-1, 1} + D_{n-1, 1} \cdot \frac{p}{2}\% = D_{n-1,1} \left( 1 + \frac{p}{2}\% \right)
\end{cases}

reformulating those equations to deposit amounts $D_i$ that interest us:

\begin{cases}
    D_0 & = D_{0,0} \\
    D_1 & = D_{1,0} = D_{0,1} \left( 1 + \frac{p}{2}\% \right) = D_{0,0} \left( 1 + \frac{p}{2}\% \right)^2 = D_0 \left( 1 + \frac{p}{2}\% \right)^2 \\
    D_2 & = D_{2,0} = D_{1,1} \left( 1 + \frac{p}{2}\% \right) = D_{1,0} \left( 1 + \frac{p}{2}\% \right)^2 = D_1 \left( 1 + \frac{p}{2}\% \right)^2 \\
    \vdots \\
    D_n & = D_{n,0} = D_{n-1,1} \left( 1 + \frac{p}{2}\% \right) = D_{n-1,0} \left( 1 + \frac{p}{2}\% \right)^2 = D_{n-1} \left( 1 + \frac{p}{2}\% \right)^2
\end{cases}

We can see that for any $m$ we just need to add more equations for each period, so:

\begin{cases}
    D_{n,0} & = D_{n} \\
    D_{n,1} & = D_{n,0} + D_{n,0} \cdot \frac{p}{m}\% = D_{n,0} \left( 1 + \frac{p}{m}\% \right) \\
    D_{n,2} & = D_{n,1} + D_{n,1} \cdot \frac{p}{m}\% = D_{n,1} \left( 1 + \frac{p}{m}\% \right) \\
    \vdots \\
    D_{n,m-1} & = D_{n, m-2} + D_{n, m-2} \cdot \frac{p}{m}\% = D_{n,m-2} \left( 1 + \frac{p}{m}\% \right) \\
    D_{n+1,0} & = D_{n, m-1} + D_{n, m-1} \cdot \frac{p}{m}\% = D_{n,m-1} \left( 1 + \frac{p}{m}\% \right)
\end{cases}

substituting equations from top to the bottom we will get that $D_{n+1} = D_{n} \left( 1 + \frac{p}{m}\% \right)^m$

##### Theorem
The deposit amount, given initial deposit $D_0$ after $n$ periods with $p\%$ periodic interest rate and $m$ frequency of compounding, i.e. $D_n$ is equal to $D_0 \left( 1 + \frac{p}{m}\% \right)^{mn}$.

##### Proof by induction
For $n=1$ we have: $D_1 = D_0 \left( 1 + \frac{p}{m}\% \right)^m$ by definition.

IH: $D_n = D_0 \left( 1 + \frac{p}{m}\% \right)^{mn}$

IS: $D_{n+1} = D_n \left( 1 + \frac{p}{m}\% \right)^m = D_0 \left( 1 + \frac{p}{m}\% \right)^{mn} \left( 1 + \frac{p}{m}\% \right)^m = D_0 \left( 1 + \frac{p}{m}\% \right)^{m(n+1)}$ and that finishes the proof.


In [6]:
# Updated deposit_compound function
def deposit_compound(n: int, m: int, D_0: float, p: float):
    """Compute deposit amount after n periods.

    Args:
        n (int): period
        m (int): frequency of compounds
        D_0 (float): initial deposit
        p (float in [0,1]): periodic interest rate

    Returns:
        D_n (float): deposit amount after n periods.
    """
    return D_0 * (1 + p/m)**(n*m)

D_10 = deposit_compound(n=10, m=12, D_0=1000, p=0.05)
print(f'Initial deposit 1000 after 10 years with 5% interest and monthly frequency: {D_10:.2f}')

Initial deposit 1000 after 10 years with 5% interest and monthly frequency: 1647.01


### Compound interest calculator (with frequency)

In [7]:
def pretty_print_deposit(n, m, D_0, p):
    return f'Deposit after {n} periods with {p} interest and {m} frequency: {deposit_compound(n, m, D_0, p):.2f}'
interact(pretty_print_deposit, n=periods_widget, m=frequency_widget, D_0=deposit_widget, p=interest_widget);

interactive(children=(BoundedIntText(value=1, description='Periods:'), BoundedIntText(value=1, description='Fr…

## Compound interest with different frequencies comparison

In [15]:
def plot_compounding_comparison(D_0: float, p: float, period: Tuple[int, int] = (1, 50)):
    """Plot deposit amount as a function of periods for different compounding frequencies

    :param D_0: initial deposit
    :type D_0: float
    :param p: periodic interest rate
    :type p: float
    :param period: range for which to plot deposit amount
    :type period: Tuple[int, int]
    :return: None
    """
    M = np.array([1, 2, 3, 4, 6, 12, 365])
    periods = np.arange(*period)
    for m in M:
        plt.plot(periods, deposit_compound(n=periods, m=m, D_0=D_0, p=p))
    plt.xlabel('years')
    plt.ylabel('amount')
    plt.title(f'Initial deposit: {D_0}, annual interest rate: {p}')
    plt.legend(labels=M, title='Frequency')
    plt.show()

interact(plot_compounding_comparison, D_0=deposit_widget, p=interest_widget, period=periods_range_widget);


interactive(children=(FloatText(value=1000.0, description='Deposit:', step=10.0), FloatSlider(value=0.05, desc…

### Continuous compounding
We see from the plots, that more frequent compounding yields more, but not that much more with each step. What if we could go with frequency to the infinity? It means that we would be dividing a period into infinitesimal chunks. Does it mean, that we will get to infinite deposits?

##### Lemma
Sequence $ a_n =  \left( 1 + \frac{p}{n} \right)^n$ is increasing, i.e:
$\forall \; k < l \; \left( 1 + \frac{p}{k} \right)^k < \left( 1 + \frac{p}{l} \right)^l $

##### Proof
Let's take $k = n$ and $l = n+1$ then:

\begin{align*}
    \left( 1 + \frac{p}{n} \right)^n &< \left( 1 + \frac{p}{n+1} \right)^{n+1} \\
    \left( \frac{n+p}{n} \right)^n &< \left( \frac{n+1+p}{n+1} \right)^{n+1} \\
    \left( \frac{n+1}{n} \right)^n (n+1) &< \left( \frac{n+1+p}{n+p} \right)^{n} (n+1+p) \\
    \left( 1 + \frac{1}{n} \right)^n (n+1) &< \left( 1 + \frac{1}{n+p} \right)^{n} (n+1+p) < \left( 1 + \frac{1}{n} \right)^n (n+1+p) \\
    0 &< p
\end{align*}

As long as $p$ is positive then the inequality is true.

##### Corollary
The more frequent compounding the more we get from the deposit.

##### Proof
Let $k, l$ be compound frequencies and $k<l$, then from the lemma we have:
$D_0 \left( 1 + \frac{p}{k}\% \right)^{kn} < D_0 \left( 1 + \frac{p}{l}\% \right)^{ln}.$

##### Theorem
The sequence $a_n$ is bounded from above for any $p$. If sequence is increasing and bounded from above then it has a limit. We call it a base of natural logarithm when $p=1$:
$\lim_{n \rightarrow \infty} \left( 1 + \frac{1}{n} \right)^n = e.$

##### Proof
Stolen from http://www.milefoot.com/math/calculus/limits/LimitDefinitionOfE10.htm
Show that $b_n = \left( 1 + \frac{1}{n} \right)^{n+1}$ is decreasing (for example look at $\frac{b_{n+1}}{b_n}$
It is obvious that $a_n < b_n$, then $a_n$ is bounded from above.

##### Corollary
The deposit amount, given initial deposit $D_0$ after time $t$ with $p\%$ periodic interest rate and continuous compounding, i.e. $D_t$ is equal to $D_0 e^{p\%t}$.

##### Proof
$D_t = \lim_{m \rightarrow \infty} D_0 \left( 1 + \frac{p}{m}\% \right)^{mt} = D_0 \left\{  \lim_{m \rightarrow \infty} \left[ \left( 1 + \frac{1}{\frac{m}{p\%}} \right)^{\frac{m}{p\%}} \right]^{p\%} \right\}^t = D_0 e^{p\%t}$